In [3]:
import pandas
import matplotlib as mpl
import xarray as xr
import numpy as np
import datetime as dt
dir_cmc='F:/data/sst/cmc/CMC0.2deg/v2/'
dir_cmc_clim='F:/data/sst/cmc/CMC0.2deg/v2/climatology/'
def get_filename(lyr,idyjl):
    d = dt.date(lyr,1,1) + dt.timedelta(idyjl - 1)
    dir_cmc='F:/data/sst/cmc/CMC0.2deg/v2/'
    syr=str(d.year).zfill(4)
    smon=str(d.month).zfill(2)
    sdym=str(d.day).zfill(2)
    sjdy=str(idyjl).zfill(3)
    fname_tem=syr + smon + sdym + '120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
    cmc_filename = dir_cmc + syr + '/' + sjdy + '/' + fname_tem
    return cmc_filename
def get_filename_v2a(lyr,idyjl):
    d = dt.date(lyr,1,1) + dt.timedelta(idyjl - 1)
    dir_cmc='F:/data/sst/cmc/CMC0.2deg/v2/'
    syr=str(d.year).zfill(4)
    smon=str(d.month).zfill(2)
    sdym=str(d.day).zfill(2)
    sjdy=str(idyjl).zfill(3)
    fname_tem=syr + smon + sdym + '120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv03.0.nc'
    cmc_filename = dir_cmc + syr + '/' + sjdy + '/' + fname_tem
    return cmc_filename
def get_filename_v2a_dir(lyr,idyjl):
    d = dt.date(lyr,1,1) + dt.timedelta(idyjl - 1)
    dir_cmc='F:/data/sst/cmc/CMC0.2deg/v2/'
    syr=str(d.year).zfill(4)
    smon=str(d.month).zfill(2)
    sdym=str(d.day).zfill(2)
    sjdy=str(idyjl).zfill(3)
    cmc_filename = dir_cmc + syr + '/' + sjdy + '/' 
    return cmc_filename
def get_filename_v3(lyr,idyjl):
    d = dt.date(lyr,1,1) + dt.timedelta(idyjl - 1)
    dir_cmc='F:/data/sst/cmc/CMC0.1deg/v3/'
    syr=str(d.year).zfill(4)
    smon=str(d.month).zfill(2)
    sdym=str(d.day).zfill(2)
    sjdy=str(idyjl).zfill(3)
    fname_tem=syr + smon + sdym + '120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc'
    cmc_filename = dir_cmc + syr + '/' + sjdy + '/' + fname_tem
    return cmc_filename

In [ ]:
#make cmc monthly average ssts
for lyr in range(1992,2017): #2017):
    ds_mnth=[]
    for imon in range(1,13):
        init = 0
        for idyjl in range(1,366):
            d = dt.date(lyr,1,1) + dt.timedelta(idyjl - 1)
            if d.month!=imon:
                continue
            cmc_filename = get_filename(lyr,idyjl)
            print(cmc_filename)
            ds = xr.open_dataset(cmc_filename,drop_variables=['analysis_error','sea_ice_fraction'])
            ds_masked = ds.where(ds['mask'] == 1.) 
            ds.close()
            ds_masked['sq_sst']=ds_masked.analysed_sst**2
            if init==0:
                ds_sum = ds_masked
                init = 1
            else:
                ds_sum = xr.concat([ds_sum,ds_masked],dim = 'time')
            print(idyjl,ds_sum.dims)
        ds_clim2 = ds_sum.resample(time='M').mean()    
#        ds_clim2 = ds_sum.groupby('time.month').mean('time')
        #ds_sum = ds_sum.mean('time',skipna=True)
        ds_mnth.append(ds_clim2)
    combined = xr.concat(ds_mnth, dim='time')
    fname_tem='monthly/' + str(lyr) + 'monthly_average_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
    cmc_filename_out = dir_cmc + fname_tem
    combined.to_netcdf(cmc_filename_out)
    

In [ ]:
#make cmc monthly average ssts FOR 0.1 deg data
#first read in 0.2 deg grid to interpolate onto

#testing 0.1 CMC
filename = 'F:/data/sst/cmc/CMC0.2deg/v2/1992/011/19920111120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
ds_v2 = xr.open_dataset(filename)
new_lat = np.linspace(ds_v2.lat[0], ds_v2.lat[-1], ds_v2.dims['lat'])
new_lon = np.linspace(ds_v2.lon[0], ds_v2.lon[-1], ds_v2.dims['lon'])

for lyr in range(2017,2019): #2017):
    ds_mnth=[]
    for imon in range(1,13):
        init = 0
        for idyjl in range(1,366):
            d = dt.date(lyr,1,1) + dt.timedelta(idyjl - 1)
            if d.month!=imon:
                continue
            cmc_filename = get_filename_v3(lyr,idyjl)
            print(cmc_filename)
            ds = xr.open_dataset(cmc_filename,drop_variables=['analysis_error','sea_ice_fraction'])
            ds_masked = ds.where(ds['mask'] == 1.) 
            ds_masked['sq_sst']=ds_masked.analysed_sst**2
            ds_interp = ds_masked.interp(lat = new_lat,lon = new_lon)            
            ds.close()
            if init==0:
                ds_sum = ds_interp
                init = 1
            else:
                ds_sum = xr.concat([ds_sum,ds_interp],dim = 'time')
            #print(idyjl,ds_sum.dims)
        ds_clim2 = ds_sum.resample(time='M').mean()    
#        ds_clim2 = ds_sum.groupby('time.month').mean('time')
        #ds_sum = ds_sum.mean('time',skipna=True)
        ds_mnth.append(ds_clim2)
    combined = xr.concat(ds_mnth, dim='time')
    fname_tem='monthly/' + str(lyr) + 'monthly_average_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
    cmc_filename_out = dir_cmc + fname_tem
    combined.to_netcdf(cmc_filename_out)
    

In [ ]:
#need to convert 2017 and 2018 to float 32 otherwise they don't combine with other index nicely
#testing data files 
file1='F:/data/sst/cmc/CMC0.2deg/v2/monthly/2016monthly_average_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
file2='F:/data/sst/cmc/CMC0.2deg/v2/monthly/2017monthly_average_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
ds1=xr.open_dataset(file1)
ds2=xr.open_dataset(file2)
ds1.close()
ds2.close()
ds3 = ds2.astype('float32')
ds3['lon']=ds1.lon
ds3['lat']=ds1.lat
file2='F:/data/sst/cmc/CMC0.2deg/v2/monthly/2017monthly_average_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
ds3.to_netcdf(file2)

#need to convert 2017 and 2018 to float 32 otherwise they don't combine with other index nicely
#testing data files 
file1='F:/data/sst/cmc/CMC0.2deg/v2/monthly/2016monthly_average_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
file2='F:/data/sst/cmc/CMC0.2deg/v2/monthly/2018monthly_average_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
ds1=xr.open_dataset(file1)
ds2=xr.open_dataset(file2)
ds1.close()
ds2.close()
ds3 = ds2.astype('float32')
ds3['lon']=ds1.lon
ds3['lat']=ds1.lat
file2='F:/data/sst/cmc/CMC0.2deg/v2/monthly/2018monthly_average_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
ds3.to_netcdf(file2)


In [ ]:
#testing data files 
file1='F:/data/sst/cmc/CMC0.2deg/v2/monthly/2016monthly_average_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
file2='F:/data/sst/cmc/CMC0.2deg/v2/monthly/2018monthly_average_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
ds1=xr.open_dataset(file1)
ds2=xr.open_dataset(file2)
ds3 = ds2.astype('float32')
ds3['lon']=ds1.lon
ds3['lat']=ds1.lat
#print(ds3)
ds1.close()
ds2.close()
ds3 = xr.concat([ds3,ds1],dim = 'time')
print(ds3)

In [3]:
#make cmc climatology from monthly files
for icase in range(0,3):
    if icase==0:
        iyr1,iyr2 = 1992,2000
    if icase==1:
        iyr1,iyr2 = 2000,2010
    if icase==2:
        iyr1,iyr2 = 2010,2019
    init = 0
    for lyr in range(iyr1,iyr2): 
        fname_tem='monthly/' + str(lyr) + 'monthly_average_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
        cmc_filename_out = dir_cmc + fname_tem
        ds = xr.open_dataset(cmc_filename_out)
        ds_masked = ds.where(ds['mask'] == 1.) 
        if init==0:
            ds_sum = ds_masked
            init = 1
        else:
            ds_sum = xr.concat([ds_sum,ds_masked],dim = 'time')
        print(lyr,ds_sum.dims)
    ds_sum2 = ds_sum.groupby('time.month').mean('time')
    fname_tem='monthly/monthly_climatology_'+str(iyr1)+'_'+str(iyr2-1)+'_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
    cmc_filename_out = dir_cmc + fname_tem
    ds_sum2.to_netcdf(cmc_filename_out)


1992 Frozen(SortedKeysDict({'time': 12, 'lat': 901, 'lon': 1800}))
1993 Frozen(SortedKeysDict({'lat': 901, 'lon': 1800, 'time': 24}))
1994 Frozen(SortedKeysDict({'lat': 901, 'lon': 1800, 'time': 36}))
1995 Frozen(SortedKeysDict({'lat': 901, 'lon': 1800, 'time': 48}))
1996 Frozen(SortedKeysDict({'lat': 901, 'lon': 1800, 'time': 60}))
1997 Frozen(SortedKeysDict({'lat': 901, 'lon': 1800, 'time': 72}))
1998 Frozen(SortedKeysDict({'lat': 901, 'lon': 1800, 'time': 84}))
1999 Frozen(SortedKeysDict({'lat': 901, 'lon': 1800, 'time': 96}))


C:\Users\gentemann\Miniconda3\envs\satenv\lib\site-packages\xarray\core\nanops.py:159: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


2000 Frozen(SortedKeysDict({'time': 12, 'lat': 901, 'lon': 1800}))
2001 Frozen(SortedKeysDict({'lat': 901, 'lon': 1800, 'time': 24}))
2002 Frozen(SortedKeysDict({'lat': 901, 'lon': 1800, 'time': 36}))
2003 Frozen(SortedKeysDict({'lat': 901, 'lon': 1800, 'time': 48}))
2004 Frozen(SortedKeysDict({'lat': 901, 'lon': 1800, 'time': 60}))
2005 Frozen(SortedKeysDict({'lat': 901, 'lon': 1800, 'time': 72}))
2006 Frozen(SortedKeysDict({'lat': 901, 'lon': 1800, 'time': 84}))
2007 Frozen(SortedKeysDict({'lat': 901, 'lon': 1800, 'time': 96}))
2008 Frozen(SortedKeysDict({'lat': 901, 'lon': 1800, 'time': 108}))
2009 Frozen(SortedKeysDict({'lat': 901, 'lon': 1800, 'time': 120}))


C:\Users\gentemann\Miniconda3\envs\satenv\lib\site-packages\xarray\core\nanops.py:159: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


2010 Frozen(SortedKeysDict({'time': 12, 'lat': 901, 'lon': 1800}))
2011 Frozen(SortedKeysDict({'lat': 901, 'lon': 1800, 'time': 24}))
2012 Frozen(SortedKeysDict({'lat': 901, 'lon': 1800, 'time': 36}))
2013 Frozen(SortedKeysDict({'lat': 901, 'lon': 1800, 'time': 48}))
2014 Frozen(SortedKeysDict({'lat': 901, 'lon': 1800, 'time': 60}))
2015 Frozen(SortedKeysDict({'lat': 901, 'lon': 1800, 'time': 72}))
2016 Frozen(SortedKeysDict({'lat': 901, 'lon': 1800, 'time': 84}))
2017 Frozen(SortedKeysDict({'lat': 901, 'lon': 1800, 'time': 96}))
2018 Frozen(SortedKeysDict({'lat': 901, 'lon': 1800, 'time': 108}))


C:\Users\gentemann\Miniconda3\envs\satenv\lib\site-packages\xarray\core\nanops.py:159: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [4]:
num_year = 2018-1992+1
num_year_file1 = 1999 - 1992 +1
num_year_file2 = 2009 - 2000 +1 
num_year_file3 = 2018 - 2010 +1
frac_file1 = num_year_file1 / num_year
frac_file2 = num_year_file2 / num_year
frac_file3 = num_year_file3 / num_year
fname_tem=dir_cmc + 'monthly/monthly_climatology_1992_1999_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
print(fname_tem)
ds = xr.open_dataset(fname_tem)
ds_masked = ds.where(ds['mask'] == 1.) 
ds.close()
fname_tem=dir_cmc + 'monthly/monthly_climatology_2000_2009_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
print(fname_tem)
ds = xr.open_dataset(fname_tem)
ds_masked2 = ds.where(ds['mask'] == 1.) 
#ds_sum = xr.concat([ds_masked2,ds_masked],dim = 'time')
ds.close()
fname_tem=dir_cmc + 'monthly/monthly_climatology_2010_2018_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
print(fname_tem)
ds = xr.open_dataset(fname_tem)
ds_masked3 = ds.where(ds['mask'] == 1.) 
#ds_sum = xr.concat([ds_masked2,ds_masked],dim = 'time')
ds.close()
ds_ave = frac_file1*ds_masked + frac_file2*ds_masked2 + frac_file3*ds_masked3
#ds_sum = ds_sum.mean('time',skipna=True)
print(frac_file1+frac_file2+frac_file3,frac_file1,frac_file2,frac_file3)
fname_tem=dir_cmc + 'monthly/monthly_climatology_1992_2018_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
print(fname_tem)
ds_ave.to_netcdf(fname_tem)

F:/data/sst/cmc/CMC0.2deg/v2/monthly/monthly_climatology_1992_1999_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc
F:/data/sst/cmc/CMC0.2deg/v2/monthly/monthly_climatology_2000_2009_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc
F:/data/sst/cmc/CMC0.2deg/v2/monthly/monthly_climatology_2010_2018_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc
1.0 0.2962962962962963 0.37037037037037035 0.3333333333333333
F:/data/sst/cmc/CMC0.2deg/v2/monthly/monthly_climatology_1992_2018_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc


In [5]:
num_year = 2018-2000+1
#num_year_file1 = 1999 - 1992 +1
num_year_file2 = 2009 - 2000 +1 
num_year_file3 = 2018 - 2010 +1
#frac_file1 = num_year_file1 / num_year
frac_file2 = num_year_file2 / num_year
frac_file3 = num_year_file3 / num_year
#fname_tem=dir_cmc + 'monthly/monthly_climatology_1992_1999_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
#print(fname_tem)
#ds = xr.open_dataset(fname_tem)
#ds_masked = ds.where(ds['mask'] == 1.) 
#ds.close()
fname_tem=dir_cmc + 'monthly/monthly_climatology_2000_2009_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
print(fname_tem)
ds = xr.open_dataset(fname_tem)
ds_masked2 = ds.where(ds['mask'] == 1.) 
#ds_sum = xr.concat([ds_masked2,ds_masked],dim = 'time')
ds.close()
fname_tem=dir_cmc + 'monthly/monthly_climatology_2010_2018_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
print(fname_tem)
ds = xr.open_dataset(fname_tem)
ds_masked3 = ds.where(ds['mask'] == 1.) 
#ds_sum = xr.concat([ds_masked2,ds_masked],dim = 'time')
ds.close()
ds_ave = frac_file2*ds_masked2 + frac_file3*ds_masked3
#ds_sum = ds_sum.mean('time',skipna=True)
print(frac_file2+frac_file3,frac_file2,frac_file3)
fname_tem=dir_cmc + 'monthly/monthly_climatology_2000_2018_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
print(fname_tem)
ds_ave.to_netcdf(fname_tem)

F:/data/sst/cmc/CMC0.2deg/v2/monthly/monthly_climatology_2000_2009_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc
F:/data/sst/cmc/CMC0.2deg/v2/monthly/monthly_climatology_2010_2018_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc
1.0 0.5263157894736842 0.47368421052631576
F:/data/sst/cmc/CMC0.2deg/v2/monthly/monthly_climatology_2000_2018_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc


In [8]:
#make daily 0.2 deg cmc files
#make cmc monthly average ssts FOR 0.1 deg data
#first read in 0.2 deg grid to interpolate onto

#20170101120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc
import os
#testing 0.1 CMC
filename = 'F:/data/sst/cmc/CMC0.2deg/v2/1992/011/19920111120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
ds_v2 = xr.open_dataset(filename)
new_lat = np.linspace(ds_v2.lat[0], ds_v2.lat[-1], ds_v2.dims['lat'])
new_lon = np.linspace(ds_v2.lon[0], ds_v2.lon[-1], ds_v2.dims['lon'])

for lyr in range(2018,2019): #2017):
    for idyjl in range(1,366):
        cmc_filename = get_filename_v3(lyr,idyjl)
        print(cmc_filename)
        ds = xr.open_dataset(cmc_filename,drop_variables=['analysis_error','sea_ice_fraction'])
        ds_masked = ds.where(ds['mask'] == 1.) 
        ds_interp = ds_masked.interp(lat = new_lat,lon = new_lon)            
        ds.close()
        cmc_filename_out = get_filename_v2a(lyr,idyjl)
        cmc_dir = get_filename_v2a_dir(lyr,idyjl)
        if not os.path.exists(cmc_dir):
            os.mkdir(cmc_dir)
        ds.to_netcdf(cmc_filename_out)

F:/data/sst/cmc/CMC0.1deg/v3/2018/001/20180101120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/002/20180102120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/003/20180103120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/004/20180104120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/005/20180105120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/006/20180106120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/007/20180107120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/008/20180108120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/009/20180109120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/010/20180110120000-CM

F:/data/sst/cmc/CMC0.1deg/v3/2018/080/20180321120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/081/20180322120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/082/20180323120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/083/20180324120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/084/20180325120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/085/20180326120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/086/20180327120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/087/20180328120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/088/20180329120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/089/20180330120000-CM

F:/data/sst/cmc/CMC0.1deg/v3/2018/159/20180608120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/160/20180609120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/161/20180610120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/162/20180611120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/163/20180612120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/164/20180613120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/165/20180614120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/166/20180615120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/167/20180616120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/168/20180617120000-CM

F:/data/sst/cmc/CMC0.1deg/v3/2018/238/20180826120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/239/20180827120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/240/20180828120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/241/20180829120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/242/20180830120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/243/20180831120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/244/20180901120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/245/20180902120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/246/20180903120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/247/20180904120000-CM

F:/data/sst/cmc/CMC0.1deg/v3/2018/317/20181113120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/318/20181114120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/319/20181115120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/320/20181116120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/321/20181117120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/322/20181118120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/323/20181119120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/324/20181120120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/325/20181121120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc
F:/data/sst/cmc/CMC0.1deg/v3/2018/326/20181122120000-CM